# 컴피티션 링크
- https://www.kaggle.com/t/2e45abe9f1434b59a3358365432a48bb

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

- 데이터 경로 변수

In [2]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/01 Practice/00_data/"
DATA_PATH

'C:/Users/user/Desktop/데이터분석/01 Practice/00_data/'

- 데이터 불러오기

In [3]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

# 특성 공학(Feature Engineering)

## 날짜 형식으로 변환

In [4]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])

## 새로 만든 feature와 병합할 고객ID로만 이루어진 데이터프레임 생성

In [5]:
train_ft = train_target[["ID"]]
test_ft = submit[["ID"]]

train_ft.shape, test_ft.shape

((14940, 1), (12225, 1))

## 구매일시를 이용한 특성생성

In [6]:
agg_list = [
        # 컬럼명, 집계 방식
        ('내점일수', lambda x: x.dt.date.nunique()),       # 내점일수 수정 : 날짜만 선택해서 nuique
        ('구매주기', lambda x: int( (x.max() - x.min()).days / x.dt.date.nunique()) ),
        ('주말방문비율', lambda x: np.mean(x.dt.weekday>4)),
        ('봄_구매비율', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
        ('여름_구매비율', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
        ('가을_구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
        ('겨울_구매비율', lambda x: np.mean(x.dt.month.isin([1,2,12]))),
        ('주구매요일', lambda x: x.dt.weekday.mode()[0]),
        ('일별평균구매건수', lambda x:  x.count() / x.dt.date.nunique() ),
        ('거래개월수', lambda x: x.dt.date.astype(str).str[:-3].nunique() ),
        
        ('아침_구매비율', lambda x: np.mean(x.dt.hour.isin(range(6, 12)))),       # 시간대 별 구매비율 추가
        ('점심_구매비율', lambda x: np.mean(x.dt.hour.isin(range(12, 18)))),      # 시간대 별 구매비율 추가
        ('저녁_구매비율', lambda x:np.mean(x.dt.hour.isin(range(18, 22)))),       # 시간대 별 구매비율 추가
        ('야간_구매비율', lambda x:np.mean(~x.dt.hour.isin(range(6, 22)))),       # 시간대 별 구매비율 추가

        ('아침_구매건수', lambda x: np.sum(x.dt.hour.isin(range(6, 12)))),       # 시간대 별 구매건수 추가
        ('점심_구매건수', lambda x: np.sum(x.dt.hour.isin(range(12, 18)))),      # 시간대 별 구매건수 추가
        ('저녁_구매건수', lambda x:np.sum(x.dt.hour.isin(range(18, 22)))),       # 시간대 별 구매건수 추가
        ('야간_구매건수', lambda x:np.sum(~x.dt.hour.isin(range(6, 22)))),       # 시간대 별 구매건수 추가

        ('월초_구매비율', lambda x: np.mean(x.dt.day <= 10)),                     # 월초 구매비율 추가(10일 이전)
        ('월말_구매비율', lambda x: np.mean(x.dt.day >= 20)),                     # 월말 구매비율 추가(20일 이후)

        ('월초_구매건수', lambda x: np.sum(x.dt.day <= 10)),                     # 월초 구매건수 추가(10일 이전)
        ('월말_구매건수', lambda x: np.sum(x.dt.day >= 20)),                     # 월말 구매건수 추가(20일 이후)

        ("주_구매_월", lambda x: x.dt.month.mode()[0]),         # 주구매 월 추가
        ("주_구매시간대", lambda x: x.dt.hour.mode()[0]),       # 주구매 시간대 추가

        
    ]

tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")

tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")

train_ft.shape, test_ft.shape

((14940, 25), (12225, 25))

- 휴면회원 여부

In [7]:
agg_list = [("최근구매일", lambda x: int( (x.max() - x.min()).days))]

sort_value = train_tr.sort_values(["ID", "구매일시"], ascending= False).drop_duplicates(subset=["ID", "구매일시"],keep='first').groupby("ID").head(2)
tmp = sort_value.groupby('ID')["구매일시"].agg(agg_list).reset_index()
tmp["휴면회원"] = (tmp["최근구매일"] > 90).astype(int)
train_ft = train_ft.merge(tmp, how= 'left', on= "ID")

sort_value = test_tr.sort_values(["ID", "구매일시"], ascending= False).drop_duplicates(subset=["ID", "구매일시"],keep='first').groupby("ID").head(2)
tmp = sort_value.groupby('ID')["구매일시"].agg(agg_list).reset_index()
tmp["휴면회원"] = (tmp["최근구매일"] > 90).astype(int)
test_ft = test_ft.merge(tmp, how= 'left', on= "ID")

train_ft.shape, test_ft.shape

((14940, 27), (12225, 27))

## 지점을 이용한 특성생성

In [8]:
agg_list = [
          ("방문지점수","nunique"),
          ('주구매지점', lambda x: x.mode()[0]),
    ]

tmp = train_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")

tmp = test_tr.groupby('ID')["지점코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")

train_ft.shape, test_ft.shape

((14940, 29), (12225, 29))

## 브랜드코드를 이용한 특성생성

In [9]:
agg_list = [
             ('브랜드코드_nunique', 'nunique'),

             ('브랜드다양성_비율',lambda x: x.nunique() / len(x)) # 브랜드다양성 추가

             ]

tmp = train_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left',on="ID")

tmp = test_tr.groupby('ID')["브랜드코드"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")

train_ft.shape, test_ft.shape

((14940, 31), (12225, 31))

## 중분류를 이용한 특성생성

In [10]:
agg_list = [
            ('중분류_nunique', 'nunique'),
            ('주구매_중분류', lambda x: x.mode()[0]),
            ]
tmp = train_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')

tmp = test_tr.groupby('ID')["중분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')

train_ft.shape, test_ft.shape

((14940, 33), (12225, 33))

## 대분류를 이용한 특성생성

In [11]:
# 새로운 대분류 범주 생성
def classification(x):
    lst_kids = ['아동_스포츠', '아동문화', '케주얼_구두_아동', '아동']
    lst_foods = ["생식품파트", "생식품"]
    lst_home = ["가정용품파트", "가정용품"]
    lst_clothes = ["스포츠캐쥬얼", "패션잡화", "여성캐쥬얼","여성캐주얼", "영어덜트캐쥬얼", "남성정장스포츠",
                   "남성의류", "여성정장", "여성의류파트", "영캐릭터","골프_유니캐쥬얼", "영플라자", "영라이브"]
    lst_loyal = ["명품잡화", "로얄부띠끄", "로얄부틱"]
    lst_daily = ["공산품", "잡화파트", "공산품파트", "잡화"]

    if x in lst_kids:
        ans = "아동"
    elif x in lst_foods:
        ans = "생식품"
    elif x in lst_home:
        ans = "가정용품" 
    elif x in lst_clothes:
        ans = "의류"      
    elif x in lst_loyal:
        ans = "명품" 
    elif x in lst_daily:
        ans = "생활용품" 
    return ans

train_tr["대분류_수정"] = train_tr["대분류"].apply(classification)
test_tr["대분류_수정"] = test_tr["대분류"].apply(classification)

train_tr.head()

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격,대분류_수정
0,train_13219,2004-05-01 09:40:00,A144000,공산품파트,차류,5100,59700,생활용품
1,train_5590,2004-05-01 09:40:00,A144000,잡화파트,화장잡화,5101,17000,생활용품
2,train_7200,2004-05-01 10:20:00,A112000,공산품,용기보증,5100,34937,생활용품
3,train_3010,2004-05-01 10:30:00,A373000,아동_스포츠,아동복,5105,19000,아동
4,train_10851,2004-05-01 10:30:00,A112000,가정용품,전화기_카세트,5110,215000,가정용품


In [12]:
agg_list = [
            ('대분류_nunique', 'nunique'),
            ('주구매_대분류', lambda x: x.mode()[0]),
            ]

tmp = train_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')

tmp = test_tr.groupby('ID')["대분류"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')

train_ft.shape, test_ft.shape

((14940, 35), (12225, 35))

In [13]:
agg_list = [
            ('대분류_수정_nunique', 'nunique'),                 # 대분류_수정 기준 구매횟수
            ('주구매_대분류_수정', lambda x: x.mode()[0]),      # 대분류_수정 기준 주구매

            ('대분류_수정_아동_구매여부', lambda x: int(x.str.contains("아동").any())),              # 대분류_수정에서 아동 구매 여부(1/0)
            ('대분류_수정_생식품_구매여부', lambda x: int(x.str.contains("생식품").any())),          # 대분류_수정에서 생식품 구매 여부(1/0)
            ('대분류_수정_가정용품_구매여부', lambda x: int(x.str.contains("가정용품").any())),      # 대분류_수정에서 가정용품 구매 여부(1/0)
            ('대분류_수정_의류_구매여부', lambda x: int(x.str.contains("의류").any())),              # 대분류_수정에서 의류 구매 여부(1/0)
            ('대분류_수정_명품_구매여부', lambda x: int(x.str.contains("명품").any())),              # 대분류_수정에서 명품 구매 여부(1/0)
            ('대분류_수정_생활용품_구매여부', lambda x: int(x.str.contains("생활용품").any())),      # 대분류_수정에서 생활용품 구매 여부(1/0)

            ('대분류_수정_아동_구매횟수', lambda x: int(x.str.contains("아동").sum())),              # 대분류_수정에서 아동 구매횟수
            ('대분류_수정_생식품_구매횟수', lambda x: int(x.str.contains("생식품").sum())),          # 대분류_수정에서 생식품 구매횟수
            ('대분류_수정_가정용품_구매횟수', lambda x: int(x.str.contains("가정용품").sum())),      # 대분류_수정에서 가정용품 구매횟수
            ('대분류_수정_의류_구매횟수', lambda x: int(x.str.contains("의류").sum())),              # 대분류_수정에서 의류 구매횟수
            ('대분류_수정_명품_구매횟수', lambda x: int(x.str.contains("명품").sum())),              # 대분류_수정에서 명품 구매횟수
            ('대분류_수정_생활용품_구매횟수', lambda x: int(x.str.contains("생활용품").sum())),      # 대분류_수정에서 생활용품 구매횟수

            ]

tmp = train_tr.groupby('ID')["대분류_수정"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')

tmp = test_tr.groupby('ID')["대분류_수정"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')

train_ft.shape, test_ft.shape

((14940, 49), (12225, 49))

## 구매가격을 이용한 특성생성

In [14]:
agg_list = [
        ('총구매액',lambda x: x[x > 0].sum()),         # 0 이상에서 적용
        ('구매건수', lambda x: x[x > 0].count()),      # 0 이상에서 적용
        ('평균구매액', lambda x: x[x > 0].mean()),     # 0 이상에서 적용
        ('최대구매액', 'max'),
        ('최소구매액',lambda x: x[x > 0].min() ) ,
        ('환불금액',lambda x: x[x < 0].sum() ) ,
        ('환불건수', lambda x: ( x < 0 ).sum() ),
        ('구매금액표준편차',lambda x: x[x>0].std() ),

        ('평균저가구매비율', lambda x: (x <= 102479.91701049241).sum() / len(x)), # 저가 구매 비율 (구매 총평균에 비례)
        ('평균고가구매비율', lambda x: (x > 102479.91701049241).sum() / len(x))   # 고가 구매 비율 (구매 총평균에 비례)


    ]

tmp = train_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left')

tmp = test_tr.groupby('ID')["구매가격"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left')

train_ft.shape, test_ft.shape

((14940, 59), (12225, 59))

- 대분류_수정 기준 금액 합계


In [15]:
for col in train_tr['대분류_수정'].unique():
    tmp = train_tr[train_tr['대분류_수정'] == col].groupby('ID')["구매가격"].sum().reset_index()
    tmp.rename(columns={"구매가격" : f"대분류_수정_{col}_구매금액"}, inplace = True)
    train_ft = train_ft.merge(tmp, on= "ID", how= "left")
    train_ft[f"대분류_수정_{col}_구매금액"] = train_ft[f"대분류_수정_{col}_구매금액"].fillna(0)
    

for col in train_tr['대분류_수정'].unique():
    tmp = test_tr[test_tr['대분류_수정'] == col].groupby('ID')["구매가격"].sum().reset_index()
    tmp.rename(columns={"구매가격" : f"대분류_수정_{col}_구매금액"}, inplace = True)
    test_ft = test_ft.merge(tmp, on= "ID", how= "left")
    test_ft[f"대분류_수정_{col}_구매금액"] = test_ft[f"대분류_수정_{col}_구매금액"].fillna(0)

train_ft.shape, test_ft.shape

((14940, 65), (12225, 65))

- 대분류_수정 기준 구입비중


In [16]:
train_ft["대분류_수정_생활용품_구입비중"] = train_ft["대분류_수정_생활용품_구매금액"] / train_ft["총구매액"]
train_ft["대분류_수정_아동_구입비중"] = train_ft["대분류_수정_아동_구매금액"] / train_ft["총구매액"]
train_ft["대분류_수정_가정용품_구입비중"] = train_ft["대분류_수정_가정용품_구매금액"] / train_ft["총구매액"]
train_ft["대분류_수정_의류_구입비중"] = train_ft["대분류_수정_의류_구매금액"] / train_ft["총구매액"]
train_ft["대분류_수정_생식품_구입비중"] = train_ft["대분류_수정_생식품_구매금액"] / train_ft["총구매액"]
train_ft["대분류_수정_명품_구입비중"] = train_ft["대분류_수정_명품_구매금액"] / train_ft["총구매액"]

test_ft["대분류_수정_생활용품_구입비중"] = test_ft["대분류_수정_생활용품_구매금액"] / test_ft["총구매액"]
test_ft["대분류_수정_아동_구입비중"] = test_ft["대분류_수정_아동_구매금액"] / test_ft["총구매액"]
test_ft["대분류_수정_가정용품_구입비중"] = test_ft["대분류_수정_가정용품_구매금액"] / test_ft["총구매액"]
test_ft["대분류_수정_의류_구입비중"] = test_ft["대분류_수정_의류_구매금액"] / test_ft["총구매액"]
test_ft["대분류_수정_생식품_구입비중"] = test_ft["대분류_수정_생식품_구매금액"] / test_ft["총구매액"]
test_ft["대분류_수정_명품_구입비중"] = test_ft["대분류_수정_명품_구매금액"] / test_ft["총구매액"]

train_ft.shape, test_ft.shape

((14940, 71), (12225, 71))

- 등급 범주 생성

In [93]:
def grade(x):
    if x >= 10000000:
        ans = 1
    elif 5000000 <= x & x < 10000000:
        ans = 2
    elif 1000000 <= x & x < 5000000:
        ans = 3
    elif 0 <= x & x < 1000000:
        ans = 4
    return ans

train_ft["등급"] = train_ft["총구매액"].apply(grade)
test_ft["등급"] = test_ft["총구매액"].apply(grade)

train_ft.shape, test_ft.shape

((14940, 72), (12225, 72))

# 항상 확인하기
- 학습데이터와 테스트 데이터의 피처개수는 동일해야 함

In [17]:
train_ft.shape, test_ft.shape

((14940, 71), (12225, 71))

# 추출한 피처 저장하기

In [ ]:
train_ft.to_csv(f"{DATA_PATH}train_common.csv",index=False)
test_ft.to_csv(f"{DATA_PATH}test_common.csv",index=False)

In [ ]:
from flaml import AutoML
import xgboost 
from xgboost import XGBRFClassifier

auto_ml_ens = AutoML()
params = { "metric" : "macro_f1",
           "task" : "classification",
           "time_budget" : 60*15,
           "seed" : 42,
           "early_stop" : True,
           "ensemble" : {'final_estimator' : XGBRFClassifier() },    # 메타모델이 로지스틱 회귀!
           "estimator_list" : ['catboost', 'lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']  }   # 앙상블에 사용할 모델 지정
auto_ml_ens.fit(train_08, target, **params)

[flaml.automl.logger: 10-30 23:41:31] {1728} INFO - task = classification
[flaml.automl.logger: 10-30 23:41:31] {1739} INFO - Evaluation method: cv
[flaml.automl.logger: 10-30 23:41:31] {1838} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 10-30 23:41:31] {1955} INFO - List of ML learners in AutoML Run: ['catboost', 'lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl.logger: 10-30 23:41:31] {2258} INFO - iteration 0, current learner catboost
[flaml.automl.logger: 10-30 23:41:38] {2393} INFO - Estimated sufficient time budget=66241s. Estimated necessary time budget=66s.
[flaml.automl.logger: 10-30 23:41:38] {2442} INFO -  at 6.8s,	estimator catboost's best error=0.3075,	best estimator catboost's best error=0.3075
[flaml.automl.logger: 10-30 23:41:38] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 10-30 23:41:38] {2442} INFO -  at 7.1s,	estimator lgbm's best error=0.6223,	best estimator catboost's best error=0.3075
[flaml.automl

In [ ]:
pred = auto_ml_ens.predict(test_08)
pred.mean()

0.3405316973415133

In [ ]:
submit["target"] = pred

submit.to_csv("전현설_08_3.csv",index=False)